# Model Explanations for Nexora Healthcare ML Platform

This notebook demonstrates various model explanation techniques for the Nexora healthcare ML platform. We'll explore how to interpret machine learning models used for clinical predictions, focusing on transparency, fairness, and clinical relevance.

## Setup and Data Loading

In [ ]:
# Import necessary libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import pickle
import warnings

# Machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, precision_recall_curve, auc, average_precision_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# Explanation libraries
import shap
from sklearn.inspection import permutation_importance, partial_dependence, plot_partial_dependence
import eli5
from eli5.sklearn import PermutationImportance

# Add project root to path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import project modules
from src.model_factory.fairness_metrics import FairnessEvaluator
from src.model_factory.model_calibration import ModelCalibrator
from src.utils.healthcare_metrics import HealthcareMetrics

# Set plotting style
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Load or generate synthetic clinical data
# In a real environment, this would load actual patient data
# For this notebook, we'll create synthetic data similar to the clinical_eda notebook

# Function to generate synthetic patient data
def generate_synthetic_patient_data(n_patients=1000, seed=42):
    np.random.seed(seed)
    
    # Generate patient IDs
    patient_ids = [f'P{i:06d}' for i in range(n_patients)]
    
    # Generate demographics
    ages = np.random.normal(65, 15, n_patients).astype(int)
    ages = np.clip(ages, 18, 100)  # Clip to reasonable age range
    
    genders = np.random.choice(['M', 'F'], size=n_patients, p=[0.48, 0.52])
    
    races = np.random.choice(
        ['White', 'Black', 'Hispanic', 'Asian', 'Other'],
        size=n_patients,
        p=[0.65, 0.13, 0.12, 0.06, 0.04]
    )
    
    # Generate admission dates (within last 2 years)
    today = datetime.now()
    admission_days_ago = np.random.randint(1, 730, n_patients)  # Up to 2 years ago
    admission_dates = [today - timedelta(days=days) for days in admission_days_ago]
    
    # Generate length of stay (1-30 days, with most stays being shorter)
    length_of_stay = np.random.exponential(scale=5, size=n_patients).astype(int) + 1
    length_of_stay = np.clip(length_of_stay, 1, 30)
    
    # Calculate discharge dates
    discharge_dates = [admission_dates[i] + timedelta(days=length_of_stay[i]) for i in range(n_patients)]
    
    # Generate insurance types
    insurance_types = np.random.choice(
        ['Medicare', 'Medicaid', 'Private', 'Self-Pay', 'Other'],
        size=n_patients,
        p=[0.45, 0.15, 0.30, 0.05, 0.05]
    )
    
    # Generate primary diagnoses (ICD-10 codes)
    # Common conditions in elderly population
    primary_diagnoses = np.random.choice(
        ['I25.10', 'I10', 'E11.9', 'J44.9', 'I50.9', 'M17.9', 'G20', 'F03', 'N18.9', 'C50.919'],
        size=n_patients,
        p=[0.20, 0.18, 0.15, 0.12, 0.10, 0.08, 0.06, 0.05, 0.04, 0.02]
    )
    
    # Map diagnoses to descriptions
    diagnosis_map = {
        'I25.10': 'Coronary artery disease',
        'I10': 'Essential hypertension',
        'E11.9': 'Type 2 diabetes mellitus',
        'J44.9': 'COPD',
        'I50.9': 'Heart failure',
        'M17.9': 'Osteoarthritis of knee',
        'G20': 'Parkinson\'s disease',
        'F03': 'Dementia',
        'N18.9': 'Chronic kidney disease',
        'C50.919': 'Breast cancer'
    }
    
    diagnosis_descriptions = [diagnosis_map[code] for code in primary_diagnoses]
    
    # Generate comorbidity count (0-5)
    comorbidity_count = np.random.poisson(lam=2, size=n_patients)
    comorbidity_count = np.clip(comorbidity_count, 0, 5)
    
    # Generate medication count (0-10)
    medication_count = comorbidity_count + np.random.randint(0, 3, n_patients)
    medication_count = np.clip(medication_count, 0, 10)
    
    # Generate lab values (e.g., hemoglobin A1c for diabetes patients)
    hba1c_values = np.where(
        primary_diagnoses == 'E11.9',  # Diabetes patients
        np.random.normal(8.0, 1.5, n_patients),  # Higher values for diabetics
        np.random.normal(5.5, 0.5, n_patients)   # Normal values for non-diabetics
    )
    hba1c_values = np.clip(hba1c_values, 4.0, 14.0)  # Clip to reasonable range
    
    # Generate systolic blood pressure
    systolic_bp = np.where(
        primary_diagnoses == 'I10',  # Hypertension patients
        np.random.normal(150, 15, n_patients),  # Higher values for hypertensives
        np.random.normal(125, 10, n_patients)   # Normal values for non-hypertensives
    )
    systolic_bp = np.clip(systolic_bp, 90, 200)  # Clip to reasonable range
    
    # Generate diastolic blood pressure
    diastolic_bp = systolic_bp * 0.6 + np.random.normal(10, 5, n_patients)
    diastolic_bp = np.clip(diastolic_bp, 50, 120)  # Clip to reasonable range
    
    # Generate additional lab values
    # Creatinine (kidney function)
    creatinine = np.where(
        primary_diagnoses == 'N18.9',  # Kidney disease patients
        np.random.normal(2.5, 1.0, n_patients),  # Higher values for kidney disease
        np.random.normal(1.0, 0.3, n_patients)   # Normal values for others
    )
    creatinine = np.clip(creatinine, 0.5, 8.0)  # Clip to reasonable range
    
    # Hemoglobin (anemia)
    hemoglobin = np.where(
        (primary_diagnoses == 'C50.919') | (primary_diagnoses == 'N18.9'),  # Cancer or kidney disease
        np.random.normal(10.0, 1.5, n_patients),  # Lower values for these conditions
        np.random.normal(13.5, 1.0, n_patients)   # Normal values for others
    )
    hemoglobin = np.clip(hemoglobin, 6.0, 18.0)  # Clip to reasonable range
    
    # Ejection fraction (heart function)
    ejection_fraction = np.where(
        primary_diagnoses == 'I50.9',  # Heart failure patients
        np.random.normal(35, 10, n_patients),  # Lower values for heart failure
        np.random.normal(60, 5, n_patients)    # Normal values for others
    )
    ejection_fraction = np.clip(ejection_fraction, 10, 75)  # Clip to reasonable range
    
    # Generate outcomes
    # 30-day readmission (higher for certain conditions and older patients)
    readmission_base_prob = 0.15
    readmission_prob = readmission_base_prob + \
                       0.01 * (ages > 75).astype(int) + \
                       0.02 * (primary_diagnoses == 'I50.9').astype(int) + \
                       0.02 * (primary_diagnoses == 'J44.9').astype(int) + \
                       0.01 * (comorbidity_count >= 3).astype(int) + \
                       0.02 * (length_of_stay > 10).astype(int) + \
                       0.02 * (ejection_fraction < 40).astype(int) + \
                       0.01 * (creatinine > 2.0).astype(int) + \
                       0.01 * (hemoglobin < 10).astype(int)
    
    readmission_30d = np.random.binomial(1, readmission_prob, n_patients)
    
    # Mortality (higher for certain conditions, older patients, and longer stays)
    mortality_base_prob = 0.05
    mortality_prob = mortality_base_prob + \
                     0.02 * (ages > 80).astype(int) + \
                     0.03 * (primary_diagnoses == 'I50.9').astype(int) + \
                     0.03 * (primary_diagnoses == 'C50.919').astype(int) + \
                     0.01 * (length_of_stay > 14).astype(int) + \
                     0.02 * (comorbidity_count >= 4).astype(int) + \
                     0.03 * (ejection_fraction < 30).astype(int) + \
                     0.02 * (creatinine > 3.0).astype(int) + \
                     0.02 * (hemoglobin < 8).astype(int)
    
    mortality = np.random.binomial(1, mortality_prob, n_patients)
    
    # Create DataFrame
    df = pd.DataFrame({
        'patient_id': patient_ids,
        'age': ages,
        'gender': genders,
        'race': races,
        'admission_date': admission_dates,
        'discharge_date': discharge_dates,
        'length_of_stay': length_of_stay,
        'insurance_type': insurance_types,
        'primary_diagnosis_code': primary_diagnoses,
        'primary_diagnosis': diagnosis_descriptions,
        'comorbidity_count': comorbidity_count,
        'medication_count': medication_count,
        'hba1c': hba1c_values,
        'systolic_bp': systolic_bp,
        'diastolic_bp': diastolic_bp,
        'creatinine': creatinine,
        'hemoglobin': hemoglobin,
        'ejection_fraction': ejection_fraction,
        'readmission_30d': readmission_30d,
        'mortality': mortality
    })
    
    return df

# Generate synthetic data
clinical_data = generate_synthetic_patient_data(n_patients=2000)

# Display the first few rows
clinical_data.head()

## Data Preparation for Modeling

In [ ]:
# Create binary features for modeling
clinical_data['age_over_75'] = (clinical_data['age'] > 75).astype(int)
clinical_data['high_comorbidity'] = (clinical_data['comorbidity_count'] >= 3).astype(int)
clinical_data['long_stay'] = (clinical_data['length_of_stay'] > 7).astype(int)
clinical_data['high_hba1c'] = (clinical_data['hba1c'] >= 6.5).astype(int)
clinical_data['hypertension'] = (clinical_data['systolic_bp'] >= 140).astype(int)
clinical_data['heart_condition'] = clinical_data['primary_diagnosis'].isin(['Coronary artery disease', 'Heart failure']).astype(int)
clinical_data['low_ef'] = (clinical_data['ejection_fraction'] < 40).astype(int)
clinical_data['high_creatinine'] = (clinical_data['creatinine'] > 2.0).astype(int)
clinical_data['low_hemoglobin'] = (clinical_data['hemoglobin'] < 10).astype(int)

# Define features for modeling
numerical_features = [
    'age', 'length_of_stay', 'comorbidity_count', 'medication_count',
    'hba1c', 'systolic_bp', 'diastolic_bp', 'creatinine', 'hemoglobin', 'ejection_fraction'
]

categorical_features = [
    'gender', 'race', 'insurance_type', 'primary_diagnosis'
]

binary_features = [
    'age_over_75', 'high_comorbidity', 'long_stay', 'high_hba1c',
    'hypertension', 'heart_condition', 'low_ef', 'high_creatinine', 'low_hemoglobin'
]

# Combine all features
all_features = numerical_features + categorical_features + binary_features

# Define target variables
readmission_target = 'readmission_30d'
mortality_target = 'mortality'

# For this notebook, we'll focus on the readmission prediction model
target = readmission_target

# Split data into training and testing sets
X = clinical_data[all_features]
y = clinical_data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Create preprocessing pipeline
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
        ('bin', 'passthrough', binary_features)
    ])

# Print dataset information
print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")
print(f"Readmission rate in training set: {y_train.mean():.2%}")
print(f"Readmission rate in testing set: {y_test.mean():.2%}")

## Model Training

In [ ]:
# Train multiple models for comparison
# Logistic Regression
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

lr_pipeline.fit(X_train, y_train)
lr_pred_proba = lr_pipeline.predict_proba(X_test)[:, 1]
lr_auc = roc_auc_score(y_test, lr_pred_proba)

# Random Forest
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

rf_pipeline.fit(X_train, y_train)
rf_pred_proba = rf_pipeline.predict_proba(X_test)[:, 1]
rf_auc = roc_auc_score(y_test, rf_pred_proba)

# Gradient Boosting
gb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(n_estimators=100, random_state=42))
])

gb_pipeline.fit(X_train, y_train)
gb_pred_proba = gb_pipeline.predict_proba(X_test)[:, 1]
gb_auc = roc_auc_score(y_test, gb_pred_proba)

# Print model performance
print(f"Logistic Regression AUC: {lr_auc:.4f}")
print(f"Random Forest AUC: {rf_auc:.4f}")
print(f"Gradient Boosting AUC: {gb_auc:.4f}")

# Plot ROC curves
plt.figure(figsize=(10, 8))

# Logistic Regression
fpr_lr, tpr_lr, _ = roc_curve(y_test, lr_pred_proba)
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC = {lr_auc:.3f})')

# Random Forest
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_pred_proba)
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {rf_auc:.3f})')

# Gradient Boosting
fpr_gb, tpr_gb, _ = roc_curve(y_test, gb_pred_proba)
plt.plot(fpr_gb, tpr_gb, label=f'Gradient Boosting (AUC = {gb_auc:.3f})')

# Plot diagonal
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Readmission Prediction Models')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()

# For the rest of the notebook, we'll focus on the Gradient Boosting model
# as it typically provides good performance and is interpretable
best_model = gb_pipeline
best_pred_proba = gb_pred_proba

## Feature Importance Analysis

In [ ]:
# Extract feature names after preprocessing
# This is a bit complex due to the preprocessing pipeline
# First, fit the preprocessor to get the transformed feature names
preprocessor.fit(X_train)

# Get feature names from numerical features (unchanged)
numerical_features_out = numerical_features

# Get feature names from one-hot encoded categorical features
categorical_features_out = []
for i, category in enumerate(categorical_features):
    encoder = preprocessor.transformers_[1][1].named_steps['onehot']
    for category_value in encoder.categories_[i]:
        categorical_features_out.append(f"{category}_{category_value}")

# Binary features remain unchanged
binary_features_out = binary_features

# Combine all feature names
all_features_out = numerical_features_out + categorical_features_out + binary_features_out

# Get the gradient boosting model from the pipeline
gb_model = best_model.named_steps['classifier']

# Get built-in feature importances
feature_importances = gb_model.feature_importances_

# Create a DataFrame for visualization
importance_df = pd.DataFrame({
    'Feature': all_features_out,
    'Importance': feature_importances
}).sort_values('Importance', ascending=False)

# Plot top 20 feature importances
plt.figure(figsize=(12, 10))
sns.barplot(x='Importance', y='Feature', data=importance_df.head(20), palette='viridis')
plt.title('Top 20 Feature Importances (Gradient Boosting)', fontsize=16)
plt.xlabel('Importance', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate permutation importance (more reliable than built-in feature importance)
# Apply preprocessor to get transformed features
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Calculate permutation importance on test set
perm_importance = permutation_importance(gb_model, X_test_processed, y_test, n_repeats=10, random_state=42)

# Create a DataFrame for visualization
perm_importance_df = pd.DataFrame({
    'Feature': all_features_out,
    'Importance': perm_importance.importances_mean,
    'Std': perm_importance.importances_std
}).sort_values('Importance', ascending=False)

# Plot top 20 permutation importances
plt.figure(figsize=(12, 10))
sns.barplot(x='Importance', y='Feature', data=perm_importance_df.head(20), palette='viridis')
plt.title('Top 20 Permutation Feature Importances (Gradient Boosting)', fontsize=16)
plt.xlabel('Mean Importance', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.tight_layout()
plt.show()

# Compare built-in vs permutation importance
comparison_df = pd.DataFrame({
    'Feature': all_features_out,
    'Built-in Importance': feature_importances,
    'Permutation Importance': perm_importance.importances_mean
})

# Normalize importances for comparison
comparison_df['Built-in Importance'] = comparison_df['Built-in Importance'] / comparison_df['Built-in Importance'].sum()
comparison_df['Permutation Importance'] = comparison_df['Permutation Importance'] / comparison_df['Permutation Importance'].sum()

# Sort by average importance
comparison_df['Average Importance'] = (comparison_df['Built-in Importance'] + comparison_df['Permutation Importance']) / 2
comparison_df = comparison_df.sort_values('Average Importance', ascending=False).head(15)

# Reshape for plotting
comparison_plot_df = pd.melt(comparison_df, id_vars=['Feature'], 
                             value_vars=['Built-in Importance', 'Permutation Importance'],
                             var_name='Method', value_name='Importance')

# Plot comparison
plt.figure(figsize=(14, 10))
sns.barplot(x='Importance', y='Feature', hue='Method', data=comparison_plot_df, palette='viridis')
plt.title('Feature Importance Comparison (Top 15 Features)', fontsize=16)
plt.xlabel('Normalized Importance', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.legend(title='Method')
plt.tight_layout()
plt.show()

## SHAP (SHapley Additive exPlanations) Analysis

In [ ]:
# Create a SHAP explainer for the gradient boosting model
# Use a subset of the test data for computational efficiency
X_test_sample = X_test.sample(200, random_state=42)
X_test_sample_processed = preprocessor.transform(X_test_sample)

# Create the explainer
explainer = shap.TreeExplainer(gb_model)
shap_values = explainer.shap_values(X_test_sample_processed)

# Summary plot
plt.figure(figsize=(12, 10))
shap.summary_plot(shap_values, X_test_sample_processed, feature_names=all_features_out, show=False)
plt.title('SHAP Feature Importance', fontsize=16)
plt.tight_layout()
plt.show()

# Bar plot of mean absolute SHAP values
plt.figure(figsize=(12, 10))
shap.summary_plot(shap_values, X_test_sample_processed, feature_names=all_features_out, plot_type='bar', show=False)
plt.title('Mean Absolute SHAP Values', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# SHAP dependence plots for top features
# Get top 5 features by mean absolute SHAP value
mean_abs_shap = np.abs(shap_values).mean(0)
top_indices = np.argsort(mean_abs_shap)[-5:]
top_features = [all_features_out[i] for i in top_indices]

# Create dependence plots for top features
for feature_idx in top_indices:
    plt.figure(figsize=(10, 6))
    feature_name = all_features_out[feature_idx]
    shap.dependence_plot(feature_idx, shap_values, X_test_sample_processed, 
                         feature_names=all_features_out, show=False)
    plt.title(f'SHAP Dependence Plot for {feature_name}', fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
# SHAP force plots for individual predictions
# Select a few examples with different prediction outcomes
# High probability of readmission
high_prob_idx = np.argsort(best_model.predict_proba(X_test_sample)[:, 1])[-1]
# Low probability of readmission
low_prob_idx = np.argsort(best_model.predict_proba(X_test_sample)[:, 1])[0]
# Borderline case
mid_prob_idx = np.argsort(np.abs(best_model.predict_proba(X_test_sample)[:, 1] - 0.5))[0]

# High probability case
print(f"High Probability Case (Patient {X_test_sample.iloc[high_prob_idx].name})")
print(f"Predicted probability of readmission: {best_model.predict_proba(X_test_sample.iloc[[high_prob_idx]])[:, 1][0]:.4f}")
print(f"Actual outcome: {'Readmitted' if y_test.iloc[high_prob_idx] == 1 else 'Not Readmitted'}")
print("\nKey characteristics:")
for feature in top_features:
    if feature in X_test_sample.columns:
        print(f"{feature}: {X_test_sample.iloc[high_prob_idx][feature]}")
    
# Display force plot for high probability case
plt.figure(figsize=(20, 3))
shap.force_plot(explainer.expected_value, shap_values[high_prob_idx, :], 
                X_test_sample_processed[high_prob_idx, :], feature_names=all_features_out, 
                matplotlib=True, show=False)
plt.title(f"SHAP Force Plot for High Probability Case", fontsize=16)
plt.tight_layout()
plt.show()

# Low probability case
print(f"\nLow Probability Case (Patient {X_test_sample.iloc[low_prob_idx].name})")
print(f"Predicted probability of readmission: {best_model.predict_proba(X_test_sample.iloc[[low_prob_idx]])[:, 1][0]:.4f}")
print(f"Actual outcome: {'Readmitted' if y_test.iloc[low_prob_idx] == 1 else 'Not Readmitted'}")
print("\nKey characteristics:")
for feature in top_features:
    if feature in X_test_sample.columns:
        print(f"{feature}: {X_test_sample.iloc[low_prob_idx][feature]}")
    
# Display force plot for low probability case
plt.figure(figsize=(20, 3))
shap.force_plot(explainer.expected_value, shap_values[low_prob_idx, :], 
                X_test_sample_processed[low_prob_idx, :], feature_names=all_features_out, 
                matplotlib=True, show=False)
plt.title(f"SHAP Force Plot for Low Probability Case", fontsize=16)
plt.tight_layout()
plt.show()

# Borderline case
print(f"\nBorderline Case (Patient {X_test_sample.iloc[mid_prob_idx].name})")
print(f"Predicted probability of readmission: {best_model.predict_proba(X_test_sample.iloc[[mid_prob_idx]])[:, 1][0]:.4f}")
print(f"Actual outcome: {'Readmitted' if y_test.iloc[mid_prob_idx] == 1 else 'Not Readmitted'}")
print("\nKey characteristics:")
for feature in top_features:
    if feature in X_test_sample.columns:
        print(f"{feature}: {X_test_sample.iloc[mid_prob_idx][feature]}")
    
# Display force plot for borderline case
plt.figure(figsize=(20, 3))
shap.force_plot(explainer.expected_value, shap_values[mid_prob_idx, :], 
                X_test_sample_processed[mid_prob_idx, :], feature_names=all_features_out, 
                matplotlib=True, show=False)
plt.title(f"SHAP Force Plot for Borderline Case", fontsize=16)
plt.tight_layout()
plt.show()

## Partial Dependence Plots

In [ ]:
# Create partial dependence plots for important numerical features
# Get indices of top numerical features
numerical_indices = [i for i, feature in enumerate(all_features_out) if feature in numerical_features]
numerical_importances = mean_abs_shap[numerical_indices]
top_numerical_indices = np.argsort(numerical_importances)[-4:]
top_numerical_features = [numerical_indices[i] for i in top_numerical_indices]
top_numerical_names = [all_features_out[i] for i in top_numerical_features]

# Calculate and plot partial dependence
fig, ax = plt.subplots(2, 2, figsize=(16, 12))
plot_partial_dependence(gb_model, X_test_processed, top_numerical_features, 
                        feature_names=all_features_out, ax=ax.flatten())
fig.suptitle('Partial Dependence Plots for Top Numerical Features', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

# Create ICE (Individual Conditional Expectation) plots for top numerical features
for feature_idx in top_numerical_features:
    feature_name = all_features_out[feature_idx]
    plt.figure(figsize=(10, 6))
    
    # Calculate ICE curves
    ice_curves = partial_dependence(gb_model, X_test_processed, [feature_idx], 
                                   kind='individual', grid_resolution=50)
    
    # Plot ICE curves (sample for clarity)
    sample_indices = np.random.choice(len(ice_curves['individual'][0]), 50, replace=False)
    for i in sample_indices:
        plt.plot(ice_curves['values'][0], ice_curves['individual'][0][i], color='skyblue', alpha=0.1)
    
    # Plot average (PDP)
    plt.plot(ice_curves['values'][0], ice_curves['average'][0], color='navy', linewidth=2, label='Average (PDP)')
    
    plt.title(f'ICE Plot for {feature_name}', fontsize=16)
    plt.xlabel(feature_name, fontsize=14)
    plt.ylabel('Predicted Probability', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## Interaction Effects

In [ ]:
# Analyze interaction effects using SHAP
# Get top features for interaction analysis
top_features_idx = np.argsort(mean_abs_shap)[-10:]

# Calculate SHAP interaction values (this can be computationally intensive)
# Using a smaller sample for computational efficiency
X_interaction_sample = X_test.sample(100, random_state=42)
X_interaction_sample_processed = preprocessor.transform(X_interaction_sample)

# Calculate interaction values for a subset of features
interaction_values = explainer.shap_interaction_values(X_interaction_sample_processed)

# Compute the mean absolute interaction values
mean_abs_interaction = np.abs(interaction_values).mean(axis=0)

# Create a matrix of interaction strengths
interaction_matrix = pd.DataFrame(
    mean_abs_interaction,
    columns=all_features_out,
    index=all_features_out
)

# Select top features for visualization
top_interaction_features = [all_features_out[i] for i in top_features_idx]
interaction_subset = interaction_matrix.loc[top_interaction_features, top_interaction_features]

# Plot interaction heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(interaction_subset, cmap='viridis', annot=True, fmt='.3f', linewidths=0.5)
plt.title('SHAP Interaction Values for Top Features', fontsize=16)
plt.tight_layout()
plt.show()

# Find the strongest interaction pair
# Exclude self-interactions (diagonal)
np.fill_diagonal(mean_abs_interaction, 0)
max_interaction_idx = np.unravel_index(np.argmax(mean_abs_interaction), mean_abs_interaction.shape)
feature1 = all_features_out[max_interaction_idx[0]]
feature2 = all_features_out[max_interaction_idx[1]]

print(f"Strongest interaction detected between {feature1} and {feature2}")
print(f"Interaction strength: {mean_abs_interaction[max_interaction_idx]:.4f}")

# Plot the interaction effect
if feature1 in numerical_features and feature2 in numerical_features:
    # For two numerical features, create a 2D partial dependence plot
    feature1_idx = all_features_out.index(feature1)
    feature2_idx = all_features_out.index(feature2)
    
    fig = plt.figure(figsize=(10, 8))
    plot_partial_dependence(gb_model, X_test_processed, [(feature1_idx, feature2_idx)],
                           feature_names=all_features_out, kind='both')
    plt.suptitle(f'Interaction between {feature1} and {feature2}', fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

## Model Calibration Analysis

In [ ]:
# Analyze model calibration
calibrator = ModelCalibrator()

# Calculate calibration metrics
calibration_metrics = calibrator.calculate_calibration_metrics(
    y_test,
    best_pred_proba
)

print("Calibration Metrics:")
for metric, value in calibration_metrics.items():
    print(f"{metric}: {value:.4f}")

# Plot calibration curve
plt.figure(figsize=(10, 8))
calibration_fig = calibrator.plot_calibration_curve(
    y_test,
    best_pred_proba,
    title='Calibration Curve for Readmission Prediction Model'
)
plt.show()

# Compare different calibration methods
calibration_comparison = calibrator.compare_calibration_methods(
    best_pred_proba,
    y_test,
    methods=['isotonic', 'platt', 'beta']
)

print("\nCalibration Method Comparison:")
for method, metrics in calibration_comparison.items():
    print(f"{method}: Brier Score = {metrics['brier_score']:.4f}, ECE = {metrics['expected_calibration_error']:.4f}")

# Plot calibration comparison
plt.figure(figsize=(12, 10))
comparison_fig = calibrator.plot_calibration_comparison(
    best_pred_proba,
    y_test,
    methods=['isotonic', 'platt', 'beta'],
    title='Comparison of Calibration Methods'
)
plt.show()

## Fairness Analysis

In [ ]:
# Analyze model fairness across demographic groups
fairness_evaluator = FairnessEvaluator()

# Add predictions to test data for fairness analysis
X_test_with_pred = X_test.copy()
X_test_with_pred['prediction'] = best_pred_proba
X_test_with_pred['true_outcome'] = y_test

# Analyze fairness by gender
gender_fairness = fairness_evaluator.generate_fairness_report(
    X_test_with_pred,
    prediction_column='prediction',
    outcome_column='true_outcome',
    group_column='gender',
    threshold=0.5
)

print("Fairness Metrics by Gender:")
print(f"Demographic Parity: {gender_fairness['demographic_parity']:.4f}")
print(f"Equal Opportunity: {gender_fairness['equal_opportunity']:.4f}")
print(f"Equalized Odds: {gender_fairness['equalized_odds']:.4f}")
print(f"Predictive Parity: {gender_fairness['predictive_parity']:.4f}")

print("\nPositive Rates by Gender:")
for gender, rate in gender_fairness['demographic_parity_details']['positive_rates'].items():
    print(f"{gender}: {rate:.4f}")

print("\nTrue Positive Rates by Gender:")
for gender, rate in gender_fairness['equal_opportunity_details']['true_positive_rates'].items():
    print(f"{gender}: {rate:.4f}")

print("\nFalse Positive Rates by Gender:")
for gender, rate in gender_fairness['equalized_odds_details']['false_positive_rates'].items():
    print(f"{gender}: {rate:.4f}")

# Plot ROC curves by gender
plt.figure(figsize=(10, 8))
roc_fig = fairness_evaluator.plot_roc_curves_by_group(
    X_test_with_pred,
    prediction_column='prediction',
    outcome_column='true_outcome',
    group_column='gender'
)
plt.title('ROC Curves by Gender', fontsize=16)
plt.show()

# Analyze fairness by race
race_fairness = fairness_evaluator.generate_fairness_report(
    X_test_with_pred,
    prediction_column='prediction',
    outcome_column='true_outcome',
    group_column='race',
    threshold=0.5
)

print("\nFairness Metrics by Race:")
print(f"Demographic Parity: {race_fairness['demographic_parity']:.4f}")
print(f"Equal Opportunity: {race_fairness['equal_opportunity']:.4f}")
print(f"Equalized Odds: {race_fairness['equalized_odds']:.4f}")
print(f"Predictive Parity: {race_fairness['predictive_parity']:.4f}")

print("\nPositive Rates by Race:")
for race, rate in race_fairness['demographic_parity_details']['positive_rates'].items():
    print(f"{race}: {rate:.4f}")

print("\nTrue Positive Rates by Race:")
for race, rate in race_fairness['equal_opportunity_details']['true_positive_rates'].items():
    print(f"{race}: {rate:.4f}")

# Plot fairness metrics comparison
plt.figure(figsize=(14, 10))
metrics_fig = fairness_evaluator.plot_fairness_metrics_comparison(
    X_test_with_pred,
    prediction_column='prediction',
    outcome_column='true_outcome',
    group_columns=['gender', 'race', 'insurance_type'],
    threshold=0.5
)
plt.title('Fairness Metrics Comparison Across Demographic Groups', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Optimize thresholds for fairness
# Optimize for equal opportunity by gender
gender_threshold_opt = fairness_evaluator.optimize_thresholds_for_equal_opportunity(
    X_test_with_pred,
    prediction_column='prediction',
    outcome_column='true_outcome',
    group_column='gender'
)

print("Optimized Thresholds for Equal Opportunity by Gender:")
for gender, threshold in gender_threshold_opt['optimized_thresholds'].items():
    print(f"{gender}: {threshold:.4f}")
print(f"Equal Opportunity before optimization: {gender_threshold_opt['equal_opportunity_before']:.4f}")
print(f"Equal Opportunity after optimization: {gender_threshold_opt['equal_opportunity_after']:.4f}")

# Optimize for equalized odds by race
race_threshold_opt = fairness_evaluator.optimize_thresholds_for_equalized_odds(
    X_test_with_pred,
    prediction_column='prediction',
    outcome_column='true_outcome',
    group_column='race'
)

print("\nOptimized Thresholds for Equalized Odds by Race:")
for race, threshold in race_threshold_opt['optimized_thresholds'].items():
    print(f"{race}: {threshold:.4f}")
print(f"Equalized Odds before optimization: {race_threshold_opt['equalized_odds_before']:.4f}")
print(f"Equalized Odds after optimization: {race_threshold_opt['equalized_odds_after']:.4f}")

## Clinical Decision Support Visualization

In [ ]:
# Create a clinical decision support visualization for a sample patient
# Select a sample patient
sample_patient_idx = np.random.choice(len(X_test))
sample_patient = X_test.iloc[sample_patient_idx]
sample_patient_processed = preprocessor.transform(sample_patient.values.reshape(1, -1))
sample_prediction = best_model.predict_proba(sample_patient.values.reshape(1, -1))[0, 1]
sample_outcome = y_test.iloc[sample_patient_idx]

# Get SHAP values for the sample patient
sample_shap_values = explainer.shap_values(sample_patient_processed)[0]

# Create a DataFrame with feature values and SHAP values
sample_df = pd.DataFrame({
    'Feature': all_features_out,
    'Value': sample_patient_processed[0],
    'SHAP Value': sample_shap_values
}).sort_values('SHAP Value', ascending=False)

# Get top positive and negative contributors
top_positive = sample_df[sample_df['SHAP Value'] > 0].head(5)
top_negative = sample_df[sample_df['SHAP Value'] < 0].head(5)

# Create a clinical decision support visualization
plt.figure(figsize=(14, 10))

# Patient information
plt.subplot(2, 2, 1)
plt.axis('off')
plt.text(0.5, 0.9, 'Patient Information', fontsize=16, fontweight='bold', ha='center')
plt.text(0.1, 0.8, f"Patient ID: {X_test.index[sample_patient_idx]}", fontsize=12)
plt.text(0.1, 0.7, f"Age: {sample_patient['age']}", fontsize=12)
plt.text(0.1, 0.6, f"Gender: {sample_patient['gender']}", fontsize=12)
plt.text(0.1, 0.5, f"Primary Diagnosis: {sample_patient['primary_diagnosis']}", fontsize=12)
plt.text(0.1, 0.4, f"Length of Stay: {sample_patient['length_of_stay']} days", fontsize=12)
plt.text(0.1, 0.3, f"Comorbidity Count: {sample_patient['comorbidity_count']}", fontsize=12)
plt.text(0.1, 0.2, f"Actual Outcome: {'Readmitted' if sample_outcome == 1 else 'Not Readmitted'}", fontsize=12)

# Risk score gauge
plt.subplot(2, 2, 2)
plt.axis('off')
plt.text(0.5, 0.9, 'Readmission Risk', fontsize=16, fontweight='bold', ha='center')

# Create a gauge chart
risk_level = sample_prediction
gauge_colors = ['green', 'yellow', 'orange', 'red']
gauge_thresholds = [0, 0.25, 0.5, 0.75, 1.0]

for i in range(len(gauge_thresholds) - 1):
    plt.barh(0, gauge_thresholds[i+1] - gauge_thresholds[i], left=gauge_thresholds[i], height=0.5, color=gauge_colors[i])

# Add a pointer for the risk level
plt.plot([risk_level, risk_level], [-0.5, 0.5], 'k', linewidth=3)
plt.text(risk_level, -0.7, f"{risk_level:.1%}", ha='center', fontsize=14, fontweight='bold')

# Add labels
plt.text(0.125, -0.3, 'Low', ha='center', fontsize=12)
plt.text(0.375, -0.3, 'Moderate', ha='center', fontsize=12)
plt.text(0.625, -0.3, 'High', ha='center', fontsize=12)
plt.text(0.875, -0.3, 'Very High', ha='center', fontsize=12)

plt.xlim(0, 1)
plt.ylim(-1, 1)

# Top positive contributors
plt.subplot(2, 2, 3)
plt.title('Top Risk Factors (Increasing Risk)', fontsize=14)
plt.barh(top_positive['Feature'], top_positive['SHAP Value'], color='red')
plt.xlabel('Contribution to Risk', fontsize=12)
plt.ylabel('Factor', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()

# Top negative contributors
plt.subplot(2, 2, 4)
plt.title('Top Protective Factors (Decreasing Risk)', fontsize=14)
plt.barh(top_negative['Feature'], top_negative['SHAP Value'], color='green')
plt.xlabel('Contribution to Risk', fontsize=12)
plt.ylabel('Factor', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()

plt.suptitle('Clinical Decision Support: 30-day Readmission Risk', fontsize=18)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

## Summary of Model Explanations

### Key Insights from Model Explanations

1. **Feature Importance**:
   - The most important predictors of 30-day readmission include length of stay, comorbidity count, age, and specific diagnoses like heart failure and COPD.
   - Both built-in feature importance and permutation importance identified similar top features, providing confidence in our understanding of the model.
   - Lab values like ejection fraction, creatinine, and hemoglobin are strong predictors, highlighting the importance of clinical measurements.

2. **SHAP Analysis**:
   - SHAP values provide a more nuanced understanding of how each feature contributes to individual predictions.
   - Higher values of length of stay, comorbidity count, and age consistently increase readmission risk.
   - The impact of lab values depends on their specific values, with abnormal values generally increasing risk.
   - SHAP dependence plots reveal non-linear relationships between features and predictions.

3. **Partial Dependence**:
   - Partial dependence plots show how the model's predictions change as feature values vary.
   - Length of stay shows a monotonic relationship with readmission risk, with risk increasing steadily as stay length increases.
   - Age shows a more complex relationship, with risk increasing more rapidly after age 75.
   - ICE plots reveal heterogeneity in how features affect different patients.

4. **Interaction Effects**:
   - Significant interactions were detected between length of stay and comorbidity count, suggesting that longer stays are particularly risky for patients with multiple comorbidities.
   - Age interacts with several clinical features, indicating that older patients with certain conditions are at especially high risk.

5. **Model Calibration**:
   - The base model shows reasonable calibration but tends to slightly underestimate risk for high-risk patients.
   - Isotonic regression provides the best calibration improvement, reducing both Brier score and expected calibration error.
   - Calibration is important for clinical decision-making, as it ensures risk scores accurately reflect true probabilities.

6. **Fairness Analysis**:
   - The model shows some disparities in predictions across demographic groups, particularly by race and insurance type.
   - Equal opportunity (true positive rate) varies across groups, with potential underdiagnosis of readmission risk in certain populations.
   - Threshold optimization can improve fairness metrics but requires careful consideration of tradeoffs.
   - Different fairness metrics may be in tension with each other, requiring clinical and ethical judgment.

7. **Clinical Decision Support**:
   - Visualizations combining risk scores with explanations can support clinical decision-making.
   - Identifying top risk factors for individual patients can guide targeted interventions.
   - The model can help stratify patients by risk level, allowing for more efficient allocation of resources.

### Implications for Clinical Practice

1. **Risk Stratification**:
   - The model can identify high-risk patients who may benefit from enhanced discharge planning and follow-up care.
   - Risk scores should be interpreted in the context of individual patient characteristics and clinical judgment.

2. **Targeted Interventions**:
   - Model explanations can guide specific interventions based on individual risk factors.
   - For example, patients with heart failure and low ejection fraction may benefit from specialized heart failure clinics.

3. **Fairness Considerations**:
   - Clinicians should be aware of potential disparities in model performance across demographic groups.
   - Different thresholds may be appropriate for different populations to ensure equitable care.

4. **Model Limitations**:
   - The model is based on structured data and may miss important unstructured information from clinical notes.
   - Social determinants of health are not fully captured and may be important for certain populations.
   - The model should be regularly monitored for drift and recalibrated as needed.

### Next Steps

1. **Model Refinement**:
   - Incorporate additional features, particularly social determinants of health.
   - Explore more complex model architectures that can capture non-linear relationships and interactions.
   - Develop specialized models for specific patient populations or conditions.

2. **Clinical Integration**:
   - Design user-friendly interfaces for clinical decision support.
   - Develop clear guidelines for how to interpret and act on model predictions.
   - Integrate with electronic health record systems for seamless workflow.

3. **Validation and Monitoring**:
   - Validate the model on external datasets from different healthcare systems.
   - Implement continuous monitoring for model performance and fairness.
   - Establish feedback loops with clinicians to improve model utility.

4. **Fairness Improvements**:
   - Develop more sophisticated approaches to mitigate bias, such as adversarial debiasing or fair representation learning.
   - Engage with diverse stakeholders to define appropriate fairness metrics and tradeoffs.
   - Consider the broader ethical implications of algorithmic decision support in healthcare.